# Knowledge Representation on the Web -- RDFS tutorial

Here we'll learn the basics of RDFS (RDF Schema) and how to perform basic RDFS reasoning with rdflib (documentation [here](https://rdflib.readthedocs.io/en/stable/)) and owlrl (documentation [here](https://owl-rl.readthedocs.io/en/latest/)).

## Imports

owlrl is a library implementing basic RDFS and OWL reasoning on top of rdflib. We'll install and import its relevant symbols.

In [403]:
import sys
!{sys.executable} -m pip install rdflib owlrl

from rdflib import Graph, RDFS, RDF, URIRef, Namespace, Literal
from owlrl import DeductiveClosure, RDFS_Semantics


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


## Loading RDFS graphs

Your file `yourRDF.ttl` already contains a basic Knowledge Graph in RDF with some RDFS semantics

First, we are going to add some RDFS semantics, and inspect the graph as-is; this is also called the "asserted graph"

**Exercise 1** 
1. load yourRDF graph
2. add additional triples using the RDFS semantics: have a look [here](https://www.w3.org/TR/rdf-schema/), and use domain and range, subPropertyOf, and Class, to say more about the instances in your graph
3. print the classes in your graph
4. print the properties of a specific class in yourRDF graph
5. print all instances in yourRDF graph (all objects that have a type) 
6. explain what constitutes a vocabulary in RDF

In [404]:
def check_graph(graph):
    ''' Additional function to check if graph is empty and print graph's length.'''
    # Loop through each triple in the graph (subj, pred, obj)
    for subj, pred, obj in graph:
        # Check if there is at least one triple in the Graph
        if (subj, pred, obj) not in graph:
           print("There are no triples in the Graph")
    # Print the number of "triples" in the Graph
    print(f"The graph has {len(graph)} statements.")

In [405]:
# define graph
graph = Graph()
graph.parse("./data/yourRDF.ttl")
# define example namespace
SOLAR = Namespace("http://example.org/solarsystem/")

In [406]:
# Add triples using store's add method.
graph.add((SOLAR.CelestialBody, RDF.type, RDFS.Class))
graph.add((SOLAR.satelliteOf, RDFS.domain, SOLAR.Satellite))
graph.add((SOLAR.satelliteOf, RDFS.range, SOLAR.CelestialBody))
graph.add((SOLAR.naturalSatelliteOf, RDFS.subPropertyOf, SOLAR.satelliteOf))
graph.add((SOLAR.naturalSatelliteOf, RDFS.domain, SOLAR.NaturalSatellite))

<Graph identifier=Nc91a0290fbf1422183b1e7b194c42f69 (<class 'rdflib.graph.Graph'>)>

In [407]:
print("Total # of statements:")
check_graph(graph)
print("\n")
print(graph.serialize(format="turtle"))

Total # of statements:
The graph has 15 statements.


@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix solar: <http://example.org/solarsystem/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

solar:CelestialBody a rdfs:Class .

solar:Moon a solar:NaturalSatellite ;
    solar:naturalSatelliteOf solar:Earth .

solar:Sun a solar:Star ;
    rdfs:label "Sun" ;
    solar:hasRadius 695000 .

solar:naturalSatelliteOf rdfs:domain solar:NaturalSatellite ;
    rdfs:subPropertyOf solar:satelliteOf .

solar:Earth a solar:Planet .

solar:Planet rdfs:subClassOf solar:CelestialBody .

solar:Star rdfs:subClassOf solar:CelestialBody .

solar:satelliteOf rdfs:domain solar:Satellite ;
    rdfs:range solar:CelestialBody .

solar:NaturalSatellite rdfs:subClassOf solar:Satellite .

solar:Satellite rdfs:subClassOf solar:CelestialBody .




In [408]:
graph.serialize(destination="./data/yourRDF.ttl")

<Graph identifier=Nc91a0290fbf1422183b1e7b194c42f69 (<class 'rdflib.graph.Graph'>)>

In [409]:
# print classes
print("Classes:")
classes = set()
for subj, pred, obj in graph.triples((None, RDFS.subClassOf, None)):
    if obj not in classes:
        print(obj)
        classes.add(obj)
    if subj not in classes:
        print(subj)
        classes.add(subj)

Classes:
http://example.org/solarsystem/CelestialBody
http://example.org/solarsystem/Planet
http://example.org/solarsystem/Star
http://example.org/solarsystem/Satellite
http://example.org/solarsystem/NaturalSatellite


In [410]:
# print properties
print("Properties of the NaturalSatellite class:")
for subj, pred, obj in graph.triples((None, RDF.type, SOLAR.NaturalSatellite)):
    for s, p, o in graph.triples((subj, None, None)):
        print(p)

Properties of the NaturalSatellite class:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://example.org/solarsystem/naturalSatelliteOf


In [411]:
# print all instances
print("Instances:")
comment = ""
for subj, pred, obj in graph.triples((None, RDF.type, None)):
    if obj == RDFS.Class:
        comment = " - class"
    print(f"{subj}{comment}")
    comment = ""

Instances:
http://example.org/solarsystem/CelestialBody - class
http://example.org/solarsystem/Moon
http://example.org/solarsystem/Sun
http://example.org/solarsystem/Earth


In RDF, a **vocabulary** on the Semantic Web consists of a set of terms or "special keywords" that define concepts and relationships to represent a specific domain. These terms include classes, subclasses, properties, and instances, allowing for the description and categorization of resources. For instance, one can specify that a resource is a class, define relationships between classes (such as subclass relationships), and indicate instances of those classes using RDF statements.

## RDFS inferencing

The inference engine in owlrl is triggered by `DeductiveClosure`, which computes the closure of the graph. This requires us to specify under which semantic regime we want to perform the inference (e.g. what kind of rules under the RDFS, OWL, etc. semantics we want the reasoner to produce derivations on). For RDFS semantics we use `RDFS_Semantics` as parameter. See extra options [here](https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.html#module-owlrl)


**Exercise 2**
1. expand the graph through RDFS semantics inference
2. print how many triples the new graph has
3. print out the triples in your new graph and inspect them. 

In [412]:
# 1. expand the graph through RDFS semantics inference
expanded = Graph()
expanded.parse("./data/yourRDF.ttl")
DeductiveClosure(RDFS_Semantics).expand(expanded)

In [413]:
# 2. print how many triples the new graph has
print("Total # of statements:")
check_graph(expanded)

Total # of statements:
The graph has 58 statements.


In [414]:
# 3. print out the triples in your new graph and inspect them. 
resource = list()
for subj, pred, obj in expanded:
    if obj != RDFS.Resource:
        print(f"({subj.n3(expanded.namespace_manager)}, {pred.n3(expanded.namespace_manager)}, {obj.n3(expanded.namespace_manager)})")
    else:
        resource.append((subj, pred, obj))
print("\n")
for subj, pred, obj in resource:
    print(f"({subj.n3(expanded.namespace_manager)}, {pred.n3(expanded.namespace_manager)}, {obj.n3(expanded.namespace_manager)})")

(solar:Satellite, rdfs:subClassOf, solar:CelestialBody)
(solar:NaturalSatellite, rdfs:subClassOf, solar:CelestialBody)
(solar:Moon, rdf:type, solar:NaturalSatellite)
(solar:Sun, rdf:type, solar:CelestialBody)
(rdfs:subClassOf, rdf:type, rdf:Property)
(rdfs:domain, rdf:type, rdf:Property)
(solar:naturalSatelliteOf, rdfs:domain, solar:NaturalSatellite)
(rdfs:subClassOf, rdfs:subPropertyOf, rdfs:subClassOf)
(solar:CelestialBody, rdf:type, rdfs:Class)
(solar:Sun, solar:hasRadius, "695000"^^xsd:integer)
(solar:hasRadius, rdfs:subPropertyOf, solar:hasRadius)
(solar:satelliteOf, rdfs:subPropertyOf, solar:satelliteOf)
(solar:naturalSatelliteOf, rdfs:subPropertyOf, solar:satelliteOf)
(solar:Moon, rdf:type, solar:CelestialBody)
(rdfs:domain, rdfs:subPropertyOf, rdfs:domain)
(solar:Earth, rdf:type, solar:CelestialBody)
(solar:NaturalSatellite, rdfs:subClassOf, solar:Satellite)
(rdf:type, rdf:type, rdf:Property)
(solar:Earth, rdf:type, solar:Planet)
(rdfs:range, rdf:type, rdf:Property)
(solar:Star

## The explicit (asserted) graph vs the implicit (derived) graph, and RDF entailment

Asserted triples are those that are explicitly stated, while derived or inferred triples are those that are implicitly stated through the semantics of RDFS. 

**Exercise 3**

1. Write here code to generate a graph that contains **RDFS derived triples only** from yourRDF Knowledge Graph, not the asserted ones. See a clue on rdflib graph algebra [here](https://rdflib.readthedocs.io/en/stable/merging.html)
2. have a look at the inferred graph. Based on the RDFS semantics, explain for each triple the rule that was used to generate it.
3. Explain the concept RDF entailment, and the types of entailment RDFS can produce


In [415]:
derived = expanded - graph
derived.namespace_manager.bind('solar', SOLAR)
print("Total # of statements:")
check_graph(derived)
print("\n")
resource = list()
property = list()
for subj, pred, obj in derived:
    if obj not in [RDFS.Resource, RDF.Property]:
        print(f"({subj.n3(derived.namespace_manager)}, {pred.n3(derived.namespace_manager)}, {obj.n3(derived.namespace_manager)})")
    elif obj == RDFS.Resource:
        resource.append((subj, pred, obj))
    elif obj == RDF.Property:
        property.append((subj, pred, obj))
for subj, pred, obj in resource:
    print(f"({subj.n3(derived.namespace_manager)}, {pred.n3(derived.namespace_manager)}, {obj.n3(derived.namespace_manager)})")
for subj, pred, obj in property:
    print(f"({subj.n3(derived.namespace_manager)}, {pred.n3(derived.namespace_manager)}, {obj.n3(derived.namespace_manager)})")

Total # of statements:
The graph has 43 statements.


(solar:NaturalSatellite, rdfs:subClassOf, solar:CelestialBody)
(solar:Sun, rdf:type, solar:CelestialBody)
(solar:hasRadius, rdfs:subPropertyOf, solar:hasRadius)
(solar:satelliteOf, rdfs:subPropertyOf, solar:satelliteOf)
(solar:Moon, rdf:type, solar:CelestialBody)
(rdfs:domain, rdfs:subPropertyOf, rdfs:domain)
(solar:Earth, rdf:type, solar:CelestialBody)
(rdf:type, rdfs:subPropertyOf, rdf:type)
(rdfs:subPropertyOf, rdfs:subPropertyOf, rdfs:subPropertyOf)
(rdfs:label, rdfs:subPropertyOf, rdfs:label)
(solar:Moon, rdf:type, solar:Satellite)
(solar:CelestialBody, rdfs:subClassOf, solar:CelestialBody)
(solar:Moon, solar:satelliteOf, solar:Earth)
(rdfs:range, rdfs:subPropertyOf, rdfs:range)
(solar:naturalSatelliteOf, rdfs:subPropertyOf, solar:naturalSatelliteOf)
(rdfs:subClassOf, rdfs:subPropertyOf, rdfs:subClassOf)
(solar:naturalSatelliteOf, rdf:type, rdfs:Resource)
(solar:satelliteOf, rdf:type, rdfs:Resource)
(solar:CelestialBody, rdfs:s

### [**RDFS entailment rules:**](https://www.w3.org/TR/rdf-mt/#rules)

* **(S, P, O) => (P, rdf:type, rdf:Property)**

In [416]:
for subj, pred, obj in property:
    print(f"({subj.n3(derived.namespace_manager)}, {pred.n3(derived.namespace_manager)}, {obj.n3(derived.namespace_manager)})")

(rdfs:domain, rdf:type, rdf:Property)
(rdfs:subClassOf, rdf:type, rdf:Property)
(rdf:type, rdf:type, rdf:Property)
(rdfs:range, rdf:type, rdf:Property)
(solar:naturalSatelliteOf, rdf:type, rdf:Property)
(solar:hasRadius, rdf:type, rdf:Property)
(rdfs:subPropertyOf, rdf:type, rdf:Property)
(solar:satelliteOf, rdf:type, rdf:Property)
(rdfs:label, rdf:type, rdf:Property)


* **(S, P, O) => (S, rdf:type, rdfs:Resource)**
* **(S, P, O) => (O, rdf:type, rdfs:Resource)**
* **(S, rdf:type, rdfs:Class) => (S, rdfs:subClassOf rdfs:Resource)**

In [417]:
for subj, pred, obj in resource:
    print(f"({subj.n3(derived.namespace_manager)}, {pred.n3(derived.namespace_manager)}, {obj.n3(derived.namespace_manager)})")

(solar:naturalSatelliteOf, rdf:type, rdfs:Resource)
(solar:satelliteOf, rdf:type, rdfs:Resource)
(solar:CelestialBody, rdfs:subClassOf, rdfs:Resource)
("695000"^^xsd:integer, rdf:type, rdfs:Resource)
(rdfs:Class, rdf:type, rdfs:Resource)
(solar:Planet, rdfs:subClassOf, rdfs:Resource)
(solar:Star, rdf:type, rdfs:Resource)
("Sun", rdf:type, rdfs:Resource)
(solar:Satellite, rdfs:subClassOf, rdfs:Resource)
(solar:NaturalSatellite, rdfs:subClassOf, rdfs:Resource)
(solar:Sun, rdf:type, rdfs:Resource)
(solar:CelestialBody, rdf:type, rdfs:Resource)
(solar:Moon, rdf:type, rdfs:Resource)
(solar:Satellite, rdf:type, rdfs:Resource)
(solar:Earth, rdf:type, rdfs:Resource)
(solar:Planet, rdf:type, rdfs:Resource)
(solar:NaturalSatellite, rdf:type, rdfs:Resource)
(solar:Star, rdfs:subClassOf, rdfs:Resource)


* **(solar:NaturalSatellite, rdfs:subClassOf, solar:CelestialBody)**
because:
    * (solar:NaturalSatellite, rdfs:subClassOf, solar:Satellite)
    * (solar:Satellite, rdfs:subClassOf, solar:CelestialBody)
 
* **(solar:Moon, rdf:type, solar:Satellite)**
because:
    * (solar:Moon, rdf:type, solar:NaturalSatellite)
    * (solar:NaturalSatellite, rdfs:subClassOf, solar:Satellite)

* **(solar:Sun, rdf:type, solar:CelestialBody)**
because:
    * (solar:Sun, rdf:type, solar:Star)
    * (solar:Star, rdfs:subClassOf, solar:CelestialBody)
 
* **(solar:Moon, rdf:type, solar:CelestialBody)**
because:
    * (solar:Moon, rdf:type, solar:NaturalSatelline)
    * (solar:NaturalSatelline, rdfs:subClassOf, solar:CelestialBody)
 
* **(solar:Earth, rdf:type, solar:CelestialBody)**
because:
    * (solar:Earth, rdf:type, solar:Planet)
    * (solar:Planet, rdfs:subClassOf, solar:CelestialBody)

* **(solar:Moon, solar:satelliteOf, solar:Earth)**
because:
    * (solar:naturalSatelliteOf, rdfs:subPropertyOf, solar:satelliteOf)
    * (solar:Moon, solar:naturalSatelliteOf, solar:Earth)

* **(solar:CelestialBody, rdfs:subClassOf, solar:CelestialBody)**
because:
    * (solar:CelestialBody, rdf:type, rdfs:Class)

* **(X, rdfs:subPropertyOf, X)**
    * **(solar:hasRadius, rdfs:subPropertyOf, solar:hasRadius)**
    * **(solar:satelliteOf, rdfs:subPropertyOf, solar:satelliteOf)**
    * **(rdfs:domain, rdfs:subPropertyOf, rdfs:domain)**
    * **(rdf:type, rdfs:subPropertyOf, rdf:type)**
    * **(rdfs:subPropertyOf, rdfs:subPropertyOf, rdfs:subPropertyOf)**
    * **(rdfs:label, rdfs:subPropertyOf, rdfs:label)**
    * **(rdfs:range, rdfs:subPropertyOf, rdfs:range)**
    * **(solar:naturalSatelliteOf, rdfs:subPropertyOf, solar:naturalSatelliteOf)**
    * **(rdfs:subClassOf, rdfs:subPropertyOf, rdfs:subClassOf)**

        * because: (X, rdf:type, rdf:Property)

**RDF entailment** refers to the logical consequences or inferred statements that can be derived from a set of RDF triples based on specific semantic rules or reasoning.

**Types of entailment:**

* *Simple Entailment Rules*: RDFS includes rules for simple entailment, covering relationships such as subclass and subproperty relationships.
* *RDF Entailment Rules*: These rules handle RDF entailment, encompassing assertions related to properties, types, and literals. They ensure completeness and provide a basis for entailment in RDF.
* *RDFS Entailment Rules*: RDFS introduces additional entailment rules that go beyond simple RDF entailment. These include rules for handling subclass relationships, domain and range information, and other class-related inferences.
    * *Extensional Entailment Rules (within RDFS)*: These rules ensure entailments based on the extensional semantics, involving additional patterns and relationships between classes and properties.
* *Datatype Entailment Rules*: RDFS includes rules specific to datatype entailment. These rules deal with datatype-related information, including legal lexical forms, canonical forms, and relationships between datatypes.

## Assignment part 2: your own webapplication. 


**Exercise 4**
1. load ingredients.rdf and recipes.rdf in one graph. The graph contains types of individuals and types of relationships between them. Print all the classes and properties in the combined graph with the namespace `ind` and the `wtm` namespace/vocabulary (`http://purl.org/heals/food/`). 
2. extend the `ind` vocabulary (`http://purl.org/heals/ingredient/`) by creating a hierarchy of ingredients (**hint: http://purl.org/heals/ingredient/CoconutMilk rdf:subClassOf http://purl.org/heals/ingredient/PlantMilk**), and make these superclasses human readable by giving them labels. 
3. do the same for the `wtm` vocabulary: add a hierarchy of recipes as well as a hierarchy of properties (**hint: http://purl.org/heals/food/hasCookingTemperature rdf:subPropertyOf ...**) 
4. print the entailed triples as we did in the previous exercise
5. give three examples of how RDF semantics could aid the chefs in your restaurant 
6. which properties and classes could you add to the `wtm` and `ind` vocabularies to further describe your recipe and ingredient knowledge graph, aiding the chefs in your restaurant?  


In [418]:
# In order to extend the vocabularies you have to come up with multiple examples that follows the given hierarchy

In [419]:
# 1. load ingredients.rdf and recipes.rdf in one graph
ingredients = Graph()
recipes = Graph()
g = Graph()
ingredients.parse("./data/ingredients.rdf")
recipes.parse("./data/recipes.rdf")
g.parse("./data/ingredients.rdf")
g.parse("./data/recipes.rdf")

print("Total # of statements:")
check_graph(g)

Total # of statements:
The graph has 1299 statements.


In [420]:
IND = Namespace("http://purl.org/heals/ingredient/")
WTM = Namespace("http://purl.org/heals/food/")

In [421]:
# All classes
classes = set()
count = 0
for subj, pred, cl in g.triples((None, RDF.type, None)):
    if cl not in classes and (cl in IND or cl in WTM):
        print(f"class {cl}")
        classes.add(cl)
        count += 1
        properties = set()
        # All properties
        print(f"properties for the class:")
        for instance, p, o in g.triples((None, RDF.type, cl)):
            for s1, property, o1 in g.triples((instance, None, None)):
                if property not in properties:
                    print(property)
                    properties.add(property)
print(f"{count} classes")

class http://purl.org/heals/food/Ingredient
properties for the class:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://purl.org/heals/food/hasGluten
http://purl.org/heals/food/hasGlycemicIndex
http://purl.org/dc/terms/source
http://www.w3.org/2000/01/rdf-schema#label
http://www.w3.org/2004/02/skos/core#definition
http://www.w3.org/2004/02/skos/core#scopeNote
http://purl.org/heals/food/hasTexture
http://purl.org/heals/food/substitutesFor
http://www.w3.org/2004/02/skos/core#editorialNote
class http://purl.org/heals/food/Yeast
properties for the class:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://purl.org/heals/food/hasGluten
http://purl.org/heals/food/hasGlycemicIndex
http://purl.org/dc/terms/source
http://www.w3.org/2000/01/rdf-schema#label
http://www.w3.org/2004/02/skos/core#definition
http://www.w3.org/2004/02/skos/core#scopeNote
class http://purl.org/heals/food/ChemicalFoodProduct
properties for the class:
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://purl.or

In [422]:
# 2. extend the IND vocabulary by creating a hierarchy of ingredients
# and make these superclasses human readable by giving them labels
g.add((IND.PlantMilk, RDFS.subClassOf, ind.ingredient))
g.add((IND.PlantMilk, RDFS.label, Literal('PlantMilk')))
g.add((IND.CoconutMilk, RDFS.subClassOf, IND.PlantMilk))

g.add((IND.Meat, RDFS.subClassOf, ind.ingredient))
g.add((IND.Meat, RDFS.label, Literal('Meat')))
g.add((IND.Bacon, RDFS.subClassOf, IND.Meat))
g.add((IND.Beef, RDFS.subClassOf, IND.Meat))
g.add((IND.Chicken, RDFS.subClassOf, IND.Meat))
g.add((IND.ChuckRoast, RDFS.subClassOf, IND.Meat))
g.add((IND.Lamb, RDFS.subClassOf, IND.Meat))
g.add((IND.CornedBeef, RDFS.subClassOf, IND.Meat))
g.add((IND.GrilledChickenKabob, RDFS.subClassOf, IND.Meat))
g.add((IND.SmotheredChickenBreast, RDFS.subClassOf, IND.Meat))

g.add((IND.Sweeteners, RDFS.subClassOf, ind.ingredient))
g.add((IND.Sweeteners, RDFS.label, Literal('Sweeteners')))
g.add((IND.Brownies, RDFS.subClassOf, IND.Sweeteners))
g.add((IND.Honey, RDFS.subClassOf, IND.Sweeteners))
g.add((IND.WhiteBread, RDFS.subClassOf, IND.Sweeteners))

g.add((IND.NutsAndSeeds, RDFS.subClassOf, ind.ingredient))
g.add((IND.NutsAndSeeds, RDFS.label, Literal('NutsAndSeeds')))
g.add((IND.Walnut, RDFS.subClassOf, IND.NutsAndSeeds))
g.add((IND.AlmondBiscotti, RDFS.subClassOf, IND.NutsAndSeeds))
g.add((IND.DriedCranberry, RDFS.subClassOf, IND.NutsAndSeeds))
g.add((IND.Pecan, RDFS.subClassOf, IND.NutsAndSeeds))

g.add((IND.Liquids, RDFS.subClassOf, ind.ingredient))
g.add((IND.Liquids, RDFS.label, Literal('Liquids')))
g.add((IND.Water, RDFS.subClassOf, IND.Liquids))
g.add((IND.CoconutMilk, RDFS.subClassOf, IND.Liquids))
g.add((IND.AppleCiderVinegar, RDFS.subClassOf, IND.Liquids))
g.add((IND.OliveOil, RDFS.subClassOf, IND.Liquids))
g.add((IND.SesameOil, RDFS.subClassOf, IND.Liquids))
g.add((IND.BalsamicVinegar, RDFS.subClassOf, IND.Liquids))
g.add((IND.WhiteVinegar, RDFS.subClassOf, IND.Liquids))

<Graph identifier=N1b8c425144e3497cb7c47a0e84226c9d (<class 'rdflib.graph.Graph'>)>

In [423]:
# ss = set()
# count = 0
# for s, p, o in g.triples((None, None, None)):
#     uri = o
#     if uri not in ss and uri in WTM:
#         print(uri)
#         ss.add(uri)
#         count += 1
# #print(count)

# for s, p, o in g.triples((None, None, None)):
#     uri = p
#     if uri not in ss and uri in WTM:
#         print(uri)
#         ss.add(uri)
#         count += 1

In [424]:
for s, p, o in g.triples((None, WTM.isRecommendedForCourse, None)):
    print(o)

http://purl.org/heals/food/Dessert
http://purl.org/heals/food/Dessert
http://purl.org/heals/food/Dessert
http://purl.org/heals/food/Dessert
http://purl.org/heals/food/Dessert
http://purl.org/heals/food/Dessert
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Entree
http://purl.org/heals/food/Side
http://purl.org/heals/food/Side
http://purl.org/heals/food/Side
http://purl.org/heals/food/Snack
http://purl.org/heals/food/Appetizer
http://purl.org/heals/food/Appetizer
http://purl.org/heals/food/Appetizer
http://purl.org/heals/food/Appetizer


In [425]:
# 3. extend the WTM vocabulary
g.add((wtm.Meals, RDF.type, RDFS.Class))
g.add((wtm.Breakfast, RDFS.subClassOf, wtm.Meals))
g.add((wtm.Lunch, RDFS.subClassOf, wtm.Meals))
g.add((wtm.Dinner, RDFS.subClassOf, wtm.Meals))

g.add((wtm.Courses, RDF.type, RDFS.Class))
g.add((wtm.Snack, RDFS.subClassOf, wtm.Courses))
g.add((wtm.Entree, RDFS.subClassOf, wtm.Courses))
g.add((wtm.Appetizer, RDFS.subClassOf, wtm.Courses))
g.add((wtm.Side, RDFS.subClassOf, wtm.Courses))
g.add((wtm.Dessert, RDFS.subClassOf, wtm.Courses))

g.add((wtm.hasGluten, RDFS.subPropertyOf, wtm.hasIngredient))

g.add((wtm.procedure, RDF.type, RDF.Property))
g.add((wtm.hasCookingTemperature, RDFS.subPropertyOf, wtm.procedure))
g.add((wtm.hasCookTime, RDFS.subPropertyOf, wtm.procedure))

<Graph identifier=N1b8c425144e3497cb7c47a0e84226c9d (<class 'rdflib.graph.Graph'>)>

In [426]:
# 4. print the entailed triples
expanded = Graph()
expanded.parse("data/ingredients.rdf")
expanded.parse("data/recipes.rdf")
DeductiveClosure(RDFS_Semantics).expand(expanded)

derived = Graph()
derived = expanded - g
derived.namespace_manager.bind('ind', IND)
derived.namespace_manager.bind('wtm', WTM)

print("Total # of statements:")
check_graph(derived)
print("\n")

for subj, pred, obj in derived:
    print(f"({subj.n3(derived.namespace_manager)}, {pred.n3(derived.namespace_manager)}, {obj.n3(derived.namespace_manager)})")

Total # of statements:
The graph has 844 statements.


(ind:Peanut, wtm:hasGluten, "0"^^xsd:nonNegativeInteger)
(ind:GreenPepper, wtm:hasGluten, "0"^^xsd:nonNegativeInteger)
("gluten free flour", rdf:type, rdfs:Resource)
(ind:SaucyShepherdPie, wtm:hasCookTime, "50"^^xsd:nonNegativeInteger)
(ind:VanillaExtract, rdf:type, rdfs:Resource)
("Dictionary.com, \"cilantro.\" [Online]. Available: https://www.dictionary.com/browse/cilantro. [Accessed Nov. 10, 2018]", rdf:type, rdfs:Resource)
(dcterms:source, rdf:type, rdf:Property)
(wtm:Side, rdf:type, rdfs:Resource)
("walnut", rdf:type, rdfs:Resource)
("black peppercorn", rdf:type, rdfs:Resource)
("celsius", rdf:type, rdfs:Resource)
("kamut muffin", rdf:type, rdfs:Resource)
(ind:Cilantro, rdf:type, rdfs:Resource)
("a cake frosted with a white frosting and covered in coconut flakes, with no ingredients contain gluten", rdf:type, rdfs:Resource)
(ind:Cornstarch, wtm:hasGlycemicIndex, "0"^^xsd:nonNegativeInteger)
(ind:Spicy, rdf:type, rdfs:Resource)

**Examples of how RDF semantics could aid the chefs in a restaurant.**

RDF semantics could help chefs:
1. represent dietary restrictions and preferences. Chefs could use entailment to automatically identify suitable recipes considering customer preferences or dietary restrictions. For instance, if a customer is allergic to gluten, the system could suggest gluten-free alternatives.
2. define relationships between ingredients to specify which ingredients can substitute for each other. For example, an RDF triple could assert that "Almond substitutes Pecan". This information can be valuable for chefs when they need to accommodate guests with allergies or preferences by suggesting alternative ingredients in recipes.
3. define relationships between recipes and meal categories. For instance, an RDF triple could assert that specific recipe is recommended for lunch. Chefs can then receive suggestions for lunch menu items based on these semantic relationships, improving the menu planning process.

**Additional classes and properties for WTM vocabulary:**

* *Cuisine Class:*
WTM.Cuisine - a class representing different cuisines (e.g., Italian, Asian, Mexican).
* *Difficulty Level Class:*
WTM.DifficultyLevel - a class representing the difficulty level of a recipe (e.g., easy, intermediate, advanced).
* *Cooking Technique Class:*
WTM.CookingTechnique - a class representing specific cooking techniques (e.g., grilling, baking, sauteing).
* *Special Occasion Property:*
WTM.isRecommendedForOccasion - a property indicating recipes suitable for special occasions (e.g., holidays, celebrations).

**Additional properties for IND vocabulary:** 

* *Organic Property:*
IND.isOrganic - a property indicating whether an ingredient is organic.
* *Nutritional Information Property:*
IND.hasNutritionalInformation - a property linking ingredients to their nutritional information.
* *Halal or Kosher Property:*
IND.isHalal and ind.isKosher - properties indicating whether an ingredient is suitable for Halal or Kosher dietary preferences.
